## Problema 2
Estudiantes:
- Matías Fuentes
- Larry Uribne

In [1]:
!pip install pyspark

In [2]:
!wget https://github.com/IIC2440/Syllabus-2023-1/raw/main/Actividades/cora.zip
!unzip /content/cora.zip

--2023-06-28 20:31:30--  https://github.com/IIC2440/Syllabus-2023-1/raw/main/Actividades/cora.zip
Resolving github.com (github.com)... 20.201.28.151
Connecting to github.com (github.com)|20.201.28.151|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IIC2440/Syllabus-2023-1/main/Actividades/cora.zip [following]
--2023-06-28 20:31:30--  https://raw.githubusercontent.com/IIC2440/Syllabus-2023-1/main/Actividades/cora.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165770 (162K) [application/zip]
Saving to: ‘cora.zip.1’

cora.zip.1          100%[===================>] 161.88K  --.-KB/s    in 0.01s   

2023-06-28 20:31:30 (11.5 MB/s) - ‘cora.zip.1’ saved [165770/165770]

unzip:  can

In [3]:
from pyspark.sql import SparkSession
import pandas as pd
import math
spark = SparkSession.builder \
    .getOrCreate()
sc = spark.sparkContext

23/06/28 20:31:33 WARN Utils: Your hostname, Matiass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.162 instead (on interface en0)
23/06/28 20:31:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/28 20:31:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Recibir input de juguete

In [4]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

Generar RDDs

In [5]:
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

Formateamos las aristas

In [6]:
rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))
rdd_edges.collect()

[(1, (2, 4)),
 (1, (3, 2)),
 (1, (4, 1)),
 (3, (2, 1)),
 (3, (5, 6)),
 (4, (5, 9)),
 (4, (6, 2)),
 (7, (8, 9)),
 (5, (6, 1))]

1. Escoge el nodo inicial, este nodo tiene costo acumulado 0 y todos los demás tienen costo acumulado
infinito.

In [7]:
def init_rdd(rdd_nodes, initial_node):
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))
rdd_init = init_rdd(rdd_nodes, 1)
rdd_init.collect()

[(1, 0), (2, inf), (3, inf), (4, inf), (5, inf), (6, inf), (7, inf), (8, inf)]

2. En cada iteración, cada nodo comunica el costo acumulado a sus vecinos. Cada nodo recibe este costo,
sumado con el costo de atravesar la arista.

In [8]:
rdd_edges.join(rdd_init).collect()

[(1, ((2, 4), 0)),
 (1, ((3, 2), 0)),
 (1, ((4, 1), 0)),
 (3, ((2, 1), inf)),
 (3, ((5, 6), inf)),
 (4, ((5, 9), inf)),
 (4, ((6, 2), inf)),
 (5, ((6, 1), inf)),
 (7, ((8, 9), inf))]

In [9]:
def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
rdd_pass_msg = pass_msg(rdd_edges, rdd_init)
rdd_pass_msg.collect()

[(2, 4),
 (3, 2),
 (4, 1),
 (2, inf),
 (5, inf),
 (5, inf),
 (6, inf),
 (6, inf),
 (8, inf)]

3. Para hacer merge de todos los mensajes dejamos el mínimo de todos los costos. Así, actualizamos cada
nodo con el costo mínimo recibido solo si es menor al costo acumulado que ya tenía ese nodo.

In [10]:
def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
    
rdd_reduce_msg = reduce_msg(rdd_pass_msg)
rdd_reduce_msg.collect()

[(2, 4), (3, 2), (4, 1), (5, inf), (6, inf), (8, inf)]

In [11]:
def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))

rdd_updated_cost = update_cost(rdd_init, rdd_reduce_msg)
rdd_updated_cost.collect()

[(1, 0), (2, 4), (3, 2), (4, 1), (5, inf), (6, inf), (7, inf), (8, inf)]

4. Si en dos iteraciones el costo en llegar para cada nodo no cambia, entonces nos detenemos.

In [12]:
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 1 si no hubo cambio y 0 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] == x[1]))
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0
stop_condition(rdd_init, rdd_updated_cost)

False

Consolidar en una función

In [13]:
def init_rdd(rdd_nodes, initial_node):
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))

def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()

def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))

def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
    
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 0 si no hubo cambio y 1 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] != x[1]))
    
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0

def single_source_shortest_path(rdd_nodes, rdd_edges, initial_node, max_iterations = 10000):
    rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))

    # Nodo inicial en 0, el resto en infinito
    rdd_init = init_rdd(rdd_nodes, initial_node)
    rdd_prev_cost = rdd_init
    
    stop_counter = 0
    for i in range(max_iterations):
        rdd_pass_msg = pass_msg(rdd_edges, rdd_prev_cost)
        #rdd_pass_msg = rdd_edges.join(rdd_prev_cost).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
        rdd_reduce_msg = reduce_msg(rdd_pass_msg)
        # rdd_reduce_msg = rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
        rdd_updated_cost = update_cost(rdd_prev_cost, rdd_reduce_msg)
        #rdd_updated_cost = rdd_prev_cost.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
        if stop_condition(rdd_prev_cost, rdd_updated_cost):
            stop_counter += 1
            if stop_counter == 2:
                break
        else:
            stop_counter = 0 
        rdd_prev_cost = rdd_updated_cost
        #print("Iteracion:", i)
        #print(rdd_updated_cost.collect())
    print(f'Total iterations: {i+1}')
    return rdd_updated_cost


Ejecutar y mostrar resultados

In [14]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

initial_node = 1
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)
rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)
rdd_result.collect()

Total iterations: 4


[(1, 0), (2, 3), (3, 2), (4, 1), (5, 8), (6, 3), (7, inf), (8, inf)]

### Pruebas con Cora
Se deja el código dispuesto para hacer pruebas con este grafo, pero es muy grande y la verdad es que no llega a converger en un tiempo razonable coriéndolo con 1 máquina.

Cargar datos de Cora y formatear para la tarea

In [15]:
# citas = pd.read_csv('cora/cora.cites',sep="\t",
#                     header=None,
#                     names=["target", "source"])
# nodes = list(set(list(citas.target.values) + list(citas.source.values)))
# edges = []
# for target, source in citas.values:
#     edges.append((source, target, 1))

Ejecutar con un nodo que es miembro de la componente conectada más grande, y tiene 5 vértices de salida

In [16]:
# rdd_nodes = sc.parallelize(nodes)
# rdd_edges = sc.parallelize(edges)
# initial_node = 1105932
# rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)